In [1]:
import pandas as pd

order_df = pd.read_csv('/content/orders_tw.txt')
products_df = pd.read_csv('/content/products_tw.txt')
vendors_df = pd.read_csv('/content/vendors_tw.txt')

In [2]:
order_df.head()

,Unnamed: 0,customer_id,geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day
0,0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,11:03:01,55 days
1,1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,11:03:01,55 days
2,2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,03:13:21,35 days
3,3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,03:13:21,35 days
4,4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,03:13:21,35 days


In [3]:
products_df.head()

,Unnamed: 0,vendor_id,product_id,name,unit_price
0,0,3d421cb5,3bb423840ffe,蘿勒燻雞三明治特餐,1.0
1,1,b08c9b37,ee11c6e7b0fd,菜脯煎蛋,1.0
2,2,49956b96,f176a5e7510d,鼠來飽套餐,1.0
3,3,406bfc1d,11973201d6cc,【辣味】培根、德腸,1.0
4,4,406bfc1d,07bafdacdb9b,（紅醬）海鮮,1.0


In [4]:
vendors_df.head()

,Unnamed: 0,vendor_id,chain_id,geohash,primary_cuisine
0,1,3cb8b730,NaN,wsqqg,小吃
1,5,ecf6ea82,NaN,wsqqg,小吃
2,7,bfbdeb8a,NaN,wsqqr,台式
3,12,ae2bec5b,3823147c,wsqqg,台式
4,13,461fb860,NaN,wsqqp,歐美


In [5]:
order_df.order_day.describe()

,order_day
count,164607
unique,90
top,84 days
freq,2702


# Initial findings

In [6]:
# (1) Most Ordered Weekday
order_df['day_of_week'].value_counts()
'''
0 = Monday
1 = Tuesday
2 = Wednesday
3 = Thursday
4 = Friday
5 = Saturday
6 = Sunday
'''

'\n0 = Monday\n1 = Tuesday\n2 = Wednesday\n3 = Thursday\n4 = Friday\n5 = Saturday\n6 = Sunday\n'

In [7]:
# (2) Top cuisine of vendor
vendors_df['primary_cuisine'].value_counts()

,count
primary_cuisine,
台式,3324
歐美,1358
飲料,1227
小吃,1180
日式,810
甜點,626
中式,235
健康餐,177
東南亞,177


In [8]:
vendors_df['primary_cuisine'].describe()

,primary_cuisine
count,9506
unique,13
top,台式
freq,3324


In [9]:
# there are 6000 vegan restaurant in TW, and Foodpanda only cover 116
116/6000 * 100

1.9333333333333333

In [10]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164608 entries, 0 to 164607
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   164608 non-null  int64  
 1   customer_id  164608 non-null  object 
 2   geohash      164608 non-null  object 
 3   order_id     164608 non-null  int64  
 4   vendor_id    164608 non-null  object 
 5   product_id   164607 non-null  object 
 6   day_of_week  164607 non-null  float64
 7   order_time   164607 non-null  object 
 8   order_day    164607 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 11.3+ MB


In [11]:
# (3) Peak Order hour
order_df['order_time'] = pd.to_datetime(order_df['order_time'], format='%H:%M:%S', errors='coerce')


In [12]:
# UTC turn to TW timezone (+8 hrs)
order_df['order_hour'] = (order_df['order_time'] + pd.Timedelta(hours=8)).dt.hour

In [13]:
order_df['order_hour'].value_counts().head(6)

,count
order_hour,
18.0,17238
11.0,14437
19.0,13496
17.0,13251
12.0,12784
13.0,9806


In [14]:
# (4) Location of most customers

geo_count = order_df['geohash'].value_counts()
geo_count

,count
geohash,
wsqqm,23027
wsqqt,20463
wsqqq,16234
wsqqw,13647
wsqqk,13265
wsqqs,12846
wsqqj,11432
wsqq7,8135
wsqqh,6478


In [15]:
# Transform geohash into lat, lng
! pip install pygeohash
import pygeohash as pgh

# Decode geohash to coordinates
lat, lng = pgh.decode(geohash='wsqqm')
print('wsqqm:',lat, lng)

lat, lng = pgh.decode(geohash='wsqqt')
print('wsqqt:',lat, lng)

lat, lng = pgh.decode(geohash='wsqqs')
print('wsqqs:',lat, lng)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
wsqqm: 25.02685546875 121.53076171875
wsqqt: 25.07080078125 121.53076171875
wsqqs: 25.07080078125 121.48681640625


In [16]:
order_df.head()

,Unnamed: 0,customer_id,geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,order_hour
0,0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,55 days,19.0
1,1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,55 days,19.0
2,2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,35 days,11.0
3,3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,35 days,11.0
4,4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,35 days,11.0


In [17]:
# new_order_df = order_df[['customer_id','geohash','order_id','vendor_id','day_of_week','order_time','order_hour']]

# Merage file


In [18]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164608 entries, 0 to 164607
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   164608 non-null  int64         
 1   customer_id  164608 non-null  object        
 2   geohash      164608 non-null  object        
 3   order_id     164608 non-null  int64         
 4   vendor_id    164608 non-null  object        
 5   product_id   164607 non-null  object        
 6   day_of_week  164607 non-null  float64       
 7   order_time   164607 non-null  datetime64[ns]
 8   order_day    164607 non-null  object        
 9   order_hour   164607 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 12.6+ MB


In [19]:
order_df_left = pd.merge(order_df, vendors_df, on='vendor_id', how='left')
master_df = pd.merge(order_df_left, products_df, on='product_id', how='left')
master_df

,Unnamed: 0_x,customer_id,geohash_x,order_id,vendor_id_x,product_id,day_of_week,order_time,order_day,order_hour,Unnamed: 0_y,chain_id,geohash_y,primary_cuisine,Unnamed: 0,vendor_id_y,name,unit_price
0,0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,55 days,19.0,6886.0,20cce73f,wsqqe,台式,NaN,NaN,NaN,NaN
1,1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,55 days,19.0,6886.0,20cce73f,wsqqe,台式,17998.0,63e62dc1,五香拌麵 X2,1.0
2,2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,35 days,11.0,6952.0,NaN,wsqqe,歐美,NaN,NaN,NaN,NaN
3,3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,35 days,11.0,6952.0,NaN,wsqqe,歐美,NaN,NaN,NaN,NaN
4,4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,35 days,11.0,6952.0,NaN,wsqqe,歐美,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164603,164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,0 days,12.0,4971.0,NaN,wsqq7,台式,168992.0,9acafdad,韭菜鍋貼,1.0
164604,164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,0 days,12.0,4971.0,NaN,wsqq7,台式,29974.0,9acafdad,韭菜水餃,1.0
164605,164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,0 days,12.0,4971.0,NaN,wsqq7,台式,8249.0,9acafdad,玉米水餃,1.0
164606,164606,f2d8a7f889,wsqq7,71572,9acafdad,28fbc3a667ca,6.0,1900-01-01 04:37:41,0 days,12.0,4971.0,NaN,wsqq7,台式,49824.0,9acafdad,招牌水餃,1.0


In [20]:
# Drop column
# [customer_id, customer_geohash, order_id, vendor_id, product_id, day_of_week, order_time, order_hour, chain_id, vendors_geohash, primary_cuisine, product_name]
master_df.drop(['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0','order_day','vendor_id_y','unit_price'],axis=1, inplace=True)
master_df

,customer_id,geohash_x,order_id,vendor_id_x,product_id,day_of_week,order_time,order_hour,chain_id,geohash_y,primary_cuisine,name
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃
164606,f2d8a7f889,wsqq7,71572,9acafdad,28fbc3a667ca,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,招牌水餃


In [21]:
# Rename column
master_df.rename(columns={'geohash_x': 'customer_geohash','vendor_id_x':'vendor_id','geohash_y': 'vendor_geohash','name':'product_name'}, inplace=True)
master_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃
164606,f2d8a7f889,wsqq7,71572,9acafdad,28fbc3a667ca,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,招牌水餃


CLEAN_DF (remove duplicates order id)

In [22]:
new_copy = master_df.copy()
clean_df = new_copy.drop_duplicates(subset='order_id')
clean_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN
9,900f8054db,wsqqm,2,707f52a7,4d8dd728ec34,3.0,1900-01-01 05:34:10,13.0,NaN,wsqqm,飲料,NaN
16,5cfa9b4775,wsqqt,3,16907f39,b96dbd609ff2,4.0,1900-01-01 04:48:04,12.0,NaN,wsqqm,台式,特級番茄牛肉麵
18,babfe111b2,wsqqm,4,b74f7b4e,ab7a1439b99b,0.0,1900-01-01 08:23:54,16.0,517f552b,wsqqw,甜點,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
164596,0ea8b9d2e4,wsqqm,71568,eee0cdd5,8bc64e24edeb,1.0,1900-01-01 09:35:08,17.0,5fbd21b3,wsqqm,台式,NaN
164600,402941e145,wsqqk,71569,4237ca75,c1533e0367c1,3.0,1900-01-01 04:06:03,12.0,339d934f,wsqqk,歐美,Party Meal (Crispy Chicken 10pc)
164601,402941e145,wsqqk,71570,d6de655f,f03ea933239f,5.0,1900-01-01 04:15:25,12.0,0caffc2b,wsqqk,日式,Deep fried pork fillet
164602,402941e145,wsqqk,71571,49684091,1af294077cfe,6.0,1900-01-01 11:21:27,19.0,217ca08b,wsqqk,日式,NaN


In [23]:
new_copy[new_copy["order_id"] == 0]

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2


In [24]:
clean_df[clean_df["order_id"] == 0]

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN


# (1) Top Cuisine by order
- Insight : Low market penetration and a significant supply-demand gap in vegan food

In [25]:
# Calculate the most of cuisine on customer_id
cuisine_counts = clean_df.groupby(by='primary_cuisine').agg({'order_id': 'count'})


In [26]:
cuisine_counts.sort_values('order_id', ascending=False)

,order_id
primary_cuisine,
台式,25189
歐美,10667
小吃,8676
飲料,8241
日式,6451
甜點,4098
中式,2058
健康餐,1713
韓式,1265


In [27]:
print('素食餐廳訂單佔',23132/2000000 * 100,'%')

素食餐廳訂單佔 1.1566 %


# (2) Order Timeslot
- Insight: Dinner > Night Snacks > Afternoon Tea > Lunch > Breakfast

In [29]:
def classify_ordertime(hour):
    if 6 <= hour < 11:
        return 'breakfast'
    elif 11 <= hour < 13:
        return 'lunch'
    elif 13 <= hour < 17:
        return 'afternoon tea'
    elif 17 <= hour < 20:
        return 'dinner'
    else:
        return 'late-night snacks'

master_df['order_classification'] = master_df['order_hour'].apply(classify_ordertime)
master_df[['day_of_week','order_classification']].value_counts()

,,count
day_of_week,order_classification,
6.0,dinner,7408
5.0,dinner,6583
4.0,dinner,6311
2.0,dinner,6139
4.0,late-night snacks,6093
0.0,dinner,6016
5.0,late-night snacks,5932
1.0,dinner,5766
3.0,dinner,5762


In [30]:
master_df[['day_of_week','order_classification']].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

,,proportion
day_of_week,order_classification,
6.0,dinner,4.5%
5.0,dinner,4.0%
4.0,dinner,3.83%
2.0,dinner,3.73%
4.0,late-night snacks,3.7%
0.0,dinner,3.65%
5.0,late-night snacks,3.6%
1.0,dinner,3.5%
3.0,dinner,3.5%


In [31]:
master_df['order_classification'].value_counts()

,count
order_classification,
dinner,43985
late-night snacks,38643
afternoon tea,31777
lunch,27221
breakfast,22982


In [32]:
master_df['order_classification'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

,proportion
order_classification,
dinner,26.72%
late-night snacks,23.48%
afternoon tea,19.3%
lunch,16.54%
breakfast,13.96%


# (3) Locations


In [33]:
import json
from math import sin, cos, sqrt, atan2, radians
import pygeohash as pgh
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

with open("/content/tw_region.json", 'r', encoding="utf-8") as f:
    taiwan_data = json.load(f)

city_coords = []
for prefecture, cities in taiwan_data['台灣'].items():
    for city, info in cities.items():
        city_coords.append((prefecture, city, (info['latitude'], info['longitude'])))

def get_distance(coord1: tuple, coord2: tuple):
    R = 6373.0
    lat1, lon1 = radians(float(coord1[0])), radians(float(coord1[1]))
    lat2, lon2 = radians(float(coord2[0])), radians(float(coord2[1]))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

cache = {}
cache_2 = {}

def geoghash_prefecture(geoghash):
    if geoghash in cache:
        return cache[geoghash]

    target_coord = pgh.decode(geohash=geoghash)
    closest_region = None
    min_distance = float('inf')

    for prefecture, city, coord in city_coords:
        distance = get_distance(target_coord, coord)
        if distance < min_distance:
            min_distance = distance
            closest_region = prefecture

    cache[geoghash] = closest_region
    return closest_region


def geoghash_city(geoghash):
    if geoghash in cache_2:
        return cache_2[geoghash]

    target_coord = pgh.decode(geohash=geoghash)
    closest_city = None
    min_distance = float('inf')

    for prefecture, city, coord in city_coords:
        distance = get_distance(target_coord, coord)
        if distance < min_distance:
            min_distance = distance
            closest_city = city

    cache_2[geoghash] = closest_city
    return closest_city

In [34]:
master_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2,dinner
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼,lunch
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃,lunch
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃,lunch
164606,f2d8a7f889,wsqq7,71572,9acafdad,28fbc3a667ca,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,招牌水餃,lunch


,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)"
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)"
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)"
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)"
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164602,402941e145,wsqqk,71571,49684091,1af294077cfe,6.0,1900-01-01 11:21:27,19.0,217ca08b,wsqqk,日式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)"
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)"
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)"
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)"


In [50]:
#Transform location
master_df["customer_prefecture"] = master_df["customer_geohash"].progress_apply(geoghash_prefecture)
master_df["customer_city"] = master_df["customer_geohash"].progress_apply(geoghash_city)
master_df["vendor_prefecture"] = master_df["vendor_geohash"].progress_apply(geoghash_prefecture)
master_df["vendor_city"] = master_df["vendor_geohash"].progress_apply(geoghash_city)
master_df

100%|██████████| 164607/164607 [00:00<00:00, 1250605.55it/s]


,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164602,402941e145,wsqqk,71571,49684091,1af294077cfe,6.0,1900-01-01 11:21:27,19.0,217ca08b,wsqqk,日式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,萬華區
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區


In [43]:
# Vendor distribution
vendors_df["location"] = vendors_df["geohash"].progress_apply(geoghash_prefecture)
vendors_df["location"].value_counts()

100%|██████████| 9506/9506 [00:00<00:00, 934098.35it/s]


,count
location,
臺北市,4323
新北市,4312
桃園市,871


In [44]:
new_copy = master_df.copy()
clean_df = new_copy.drop_duplicates(subset='order_id')
clean_df["customer_prefecture"].value_counts()

,count
customer_prefecture,
臺北市,45441
新北市,24453
桃園市,1679


In [45]:
# Customer distribution in %
clean_df["customer_prefecture"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

,proportion
customer_prefecture,
臺北市,63.49%
新北市,34.17%
桃園市,2.35%


In [46]:
clean_df["customer_city"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

,proportion
customer_city,
中正區,14.59%
中山區,13.07%
信義區,10.16%
松山區,8.36%
萬華區,8.17%
三重區,7.36%
永和區,6.88%
新莊區,4.74%
士林區,3.96%


# (4) Demand-Supply ratios
Insight:
-


In [51]:
# Find lat,lng
import pygeohash as pgh

def location(input):

    lat, lng = pgh.decode(geohash = input)

    return lat, lng

master_df['customer_location'] = master_df['customer_geohash'].apply(location)
master_df['vendor_location'] = master_df['vendor_geohash'].apply(location)
master_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)"
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)"
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)"
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)"
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164602,402941e145,wsqqk,71571,49684091,1af294077cfe,6.0,1900-01-01 11:21:27,19.0,217ca08b,wsqqk,日式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,萬華區,"(25.02685546875, 121.48681640625)"
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)"
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)"
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)"


In [52]:
# Find distnace
from geopy.distance import geodesic
import time

def calculate_distance(row):
    point1 = row['customer_location']
    point2 = row['vendor_location']
    return round(geodesic(point1, point2).km, 3)

master_df['distance'] = master_df.apply(calculate_distance, axis=1)
master_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location,distance
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.0
1,900f8054db,wsqqe,0,63e62dc1,7576b2eab768,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,五香拌麵 X2,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.0
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.0
3,900f8054db,wsqqe,1,f15fbae2,1f3e3644c180,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.0
4,900f8054db,wsqqe,1,f15fbae2,8b995e631939,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164602,402941e145,wsqqk,71571,49684091,1af294077cfe,6.0,1900-01-01 11:21:27,19.0,217ca08b,wsqqk,日式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,萬華區,"(25.02685546875, 121.48681640625)",0.0
164603,f2d8a7f889,wsqq7,71572,9acafdad,22a1fd50fc27,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜鍋貼,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)",0.0
164604,f2d8a7f889,wsqq7,71572,9acafdad,3a4d6636ee68,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,韭菜水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)",0.0
164605,f2d8a7f889,wsqq7,71572,9acafdad,6fa181628458,6.0,1900-01-01 04:37:41,12.0,NaN,wsqq7,台式,玉米水餃,lunch,新北市,新莊區,"(25.02685546875, 121.44287109375)",新北市,新莊區,"(25.02685546875, 121.44287109375)",0.0


In [53]:
# Drop duplicated customer ID
m_df = master_df.drop_duplicates(subset='customer_id')
m_df

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location,distance
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
16,5cfa9b4775,wsqqt,3,16907f39,b96dbd609ff2,4.0,1900-01-01 04:48:04,12.0,NaN,wsqqm,台式,特級番茄牛肉麵,lunch,臺北市,中山區,"(25.07080078125, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",4.868
18,babfe111b2,wsqqm,4,b74f7b4e,ab7a1439b99b,0.0,1900-01-01 08:23:54,16.0,517f552b,wsqqw,甜點,NaN,afternoon tea,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,松山區,"(25.07080078125, 121.57470703125)",6.585
24,07335bed02,wsqqm,8,fbbfb8a6,5ccfb41cb3a2,0.0,1900-01-01 09:21:34,17.0,NaN,wsqqt,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中山區,"(25.07080078125, 121.53076171875)",4.868
26,162036b460,wsqqt,9,06c48514,fc582e1ff9fe,1.0,1900-01-01 13:59:47,21.0,NaN,wsqqs,台式,NaN,late-night snacks,臺北市,中山區,"(25.07080078125, 121.53076171875)",新北市,三重區,"(25.07080078125, 121.48681640625)",4.434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164580,2d23084664,wsqqk,71562,45472d51,fec1dde0a6da,0.0,1900-01-01 05:49:59,13.0,f457434d,wsqqm,甜點,NaN,afternoon tea,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,中正區,"(25.02685546875, 121.53076171875)",4.435
164582,f637908c29,wsqqm,71563,82488477,ae372b3eec8f,1.0,1900-01-01 10:46:06,18.0,NaN,wsqqq,歐美,青醬海鮮義大利麵套餐,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,信義區,"(25.02685546875, 121.57470703125)",4.435
164596,0ea8b9d2e4,wsqqm,71568,eee0cdd5,8bc64e24edeb,1.0,1900-01-01 09:35:08,17.0,5fbd21b3,wsqqm,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",0.000
164600,402941e145,wsqqk,71569,4237ca75,c1533e0367c1,3.0,1900-01-01 04:06:03,12.0,339d934f,wsqqk,歐美,Party Meal (Crispy Chicken 10pc),lunch,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,萬華區,"(25.02685546875, 121.48681640625)",0.000


In [54]:
customer_count_by_city = m_df.groupby('customer_city')['customer_id'].count().reset_index()
customer_count_by_city = customer_count_by_city.rename(columns={"customer_city": "City", "customer_id": "customer_id_count"})
display(customer_count_by_city)

,City,customer_id_count
0,三重區,1356
1,中和區,888
2,中山區,2438
3,中正區,3235
4,五股區,842
5,信義區,2120
6,內湖區,538
7,八德區,41
8,八里區,15
9,北投區,69


In [55]:
# Drop duplicated vendor ID
m_df2 = master_df.drop_duplicates(subset='vendor_id')
m_df2

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location,distance
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
2,900f8054db,wsqqe,1,f15fbae2,1452f0fd3ada,6.0,1900-01-01 03:13:21,11.0,NaN,wsqqe,歐美,NaN,lunch,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
9,900f8054db,wsqqm,2,707f52a7,4d8dd728ec34,3.0,1900-01-01 05:34:10,13.0,NaN,wsqqm,飲料,NaN,afternoon tea,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",0.000
16,5cfa9b4775,wsqqt,3,16907f39,b96dbd609ff2,4.0,1900-01-01 04:48:04,12.0,NaN,wsqqm,台式,特級番茄牛肉麵,lunch,臺北市,中山區,"(25.07080078125, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",4.868
18,babfe111b2,wsqqm,4,b74f7b4e,ab7a1439b99b,0.0,1900-01-01 08:23:54,16.0,517f552b,wsqqw,甜點,NaN,afternoon tea,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,松山區,"(25.07080078125, 121.57470703125)",6.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163851,1375859d77,wsqq5,71233,1de090a0,75ca2114dddf,6.0,1900-01-01 06:47:12,14.0,NaN,wsqq5,台式,牛肉咖哩飯套餐,afternoon tea,新北市,土城區,"(24.98291015625, 121.44287109375)",新北市,土城區,"(24.98291015625, 121.44287109375)",0.000
164081,d2e44d1fb0,wsqqs,71338,80869d14,bbdae53722a1,6.0,1900-01-01 10:56:45,18.0,e8182792,wsqqs,日式,NaN,dinner,新北市,三重區,"(25.07080078125, 121.48681640625)",新北市,三重區,"(25.07080078125, 121.48681640625)",0.000
164327,7bbd1d679b,wsqqw,71442,cf1b972f,20dcedbb7fad,3.0,1900-01-01 10:17:32,18.0,a42d229f,wsqqm,中式,蒜香脆皮雞【半隻】,dinner,臺北市,松山區,"(25.07080078125, 121.57470703125)",臺北市,中正區,"(25.02685546875, 121.53076171875)",6.585
164375,a212d4e5f0,wsqq0,71467,b7d2085c,1fb252703c48,5.0,1900-01-01 01:17:11,9.0,5a3894f2,wsqq0,歐美,NaN,breakfast,桃園市,桃園區,"(24.98291015625, 121.31103515625)",桃園市,桃園區,"(24.98291015625, 121.31103515625)",0.000


In [56]:
vendor_count_by_city = m_df2.groupby('vendor_city')['vendor_id'].count().reset_index()
vendor_count_by_city = vendor_count_by_city.rename(columns={"vendor_city": "City", "vendor_id": "vendor_id_count"})
display(vendor_count_by_city)

,City,vendor_id_count
0,三重區,724
1,中和區,388
2,中山區,791
3,中正區,918
4,五股區,413
5,信義區,619
6,內湖區,223
7,南港區,13
8,土城區,447
9,士林區,281


In [57]:
city_demand_supply = pd.merge(customer_count_by_city, vendor_count_by_city, on='City', how='outer')
city_demand_supply['vendor_id_count'] = city_demand_supply['vendor_id_count'].fillna(0)
display(city_demand_supply)

,City,customer_id_count,vendor_id_count
0,三重區,1356,724.0
1,中和區,888,388.0
2,中山區,2438,791.0
3,中正區,3235,918.0
4,五股區,842,413.0
5,信義區,2120,619.0
6,內湖區,538,223.0
7,八德區,41,0.0
8,八里區,15,0.0
9,北投區,69,0.0


In [58]:
import numpy as np
city_demand_supply['demand_supply_ratio'] = (city_demand_supply['customer_id_count'] / city_demand_supply['vendor_id_count']).round(2)
city_demand_supply['demand_supply_ratio'] = city_demand_supply['demand_supply_ratio'].replace([np.inf, -np.inf],0)
display(city_demand_supply)

,City,customer_id_count,vendor_id_count,demand_supply_ratio
0,三重區,1356,724.0,1.87
1,中和區,888,388.0,2.29
2,中山區,2438,791.0,3.08
3,中正區,3235,918.0,3.52
4,五股區,842,413.0,2.04
5,信義區,2120,619.0,3.42
6,內湖區,538,223.0,2.41
7,八德區,41,0.0,0.00
8,八里區,15,0.0,0.00
9,北投區,69,0.0,0.00


In [59]:
# city with no vendor
city_demand_supply[city_demand_supply['demand_supply_ratio'] == 0]

,City,customer_id_count,vendor_id_count,demand_supply_ratio
7,八德區,41,0.0,0.0
8,八里區,15,0.0,0.0
9,北投區,69,0.0,0.0
13,大園區,1,0.0,0.0
14,大溪區,2,0.0,0.0
15,平鎮區,4,0.0,0.0
17,新店區,163,0.0,0.0
24,汐止區,59,0.0,0.0
26,淡水區,4,0.0,0.0


## Demand supply ratio per order in peak order time-slot (dinner)

In [60]:
o_df = master_df.drop_duplicates(subset='order_id')
o_df_dinner = o_df[o_df['order_classification'] == 'dinner']
o_df_dinner

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location,distance
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
23,babfe111b2,wsqqm,7,7fff4be6,1a31e43c524d,6.0,1900-01-01 11:19:22,19.0,NaN,wsqqm,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",0.000
24,07335bed02,wsqqm,8,fbbfb8a6,5ccfb41cb3a2,0.0,1900-01-01 09:21:34,17.0,NaN,wsqqt,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中山區,"(25.07080078125, 121.53076171875)",4.868
49,162036b460,wsqqt,14,c9d99270,a858af19e1ad,2.0,1900-01-01 10:15:00,18.0,NaN,wsqqt,台式,NaN,dinner,臺北市,中山區,"(25.07080078125, 121.53076171875)",臺北市,中山區,"(25.07080078125, 121.53076171875)",0.000
73,d6e844fb2e,wsqq6,22,b30567ea,a29d4390dbaa,4.0,1900-01-01 11:15:14,19.0,463204ae,wsqq7,中式,NaN,dinner,新北市,泰山區,"(25.02685546875, 121.39892578125)",新北市,新莊區,"(25.02685546875, 121.44287109375)",4.435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164571,eb41339335,wsqqe,71556,5fb6a265,a9bb5dc52721,3.0,1900-01-01 10:28:39,18.0,c4a01c2d,wsqqe,小吃,麻辣鴨翅,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
164582,f637908c29,wsqqm,71563,82488477,ae372b3eec8f,1.0,1900-01-01 10:46:06,18.0,NaN,wsqqq,歐美,青醬海鮮義大利麵套餐,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,信義區,"(25.02685546875, 121.57470703125)",4.435
164584,f637908c29,wsqqw,71564,220c1a0b,b2c7607b1791,2.0,1900-01-01 09:35:49,17.0,NaN,wsqqw,台式,玉米鍋貼,dinner,臺北市,松山區,"(25.07080078125, 121.57470703125)",臺北市,松山區,"(25.07080078125, 121.57470703125)",0.000
164596,0ea8b9d2e4,wsqqm,71568,eee0cdd5,8bc64e24edeb,1.0,1900-01-01 09:35:08,17.0,5fbd21b3,wsqqm,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",0.000


In [61]:
v_df = master_df.drop_duplicates(subset='vendor_id')
v_df_dinner = v_df[v_df['order_classification'] == 'dinner']
v_df_dinner

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_hour,chain_id,vendor_geohash,primary_cuisine,product_name,order_classification,customer_prefecture,customer_city,customer_location,vendor_prefecture,vendor_city,vendor_location,distance
0,900f8054db,wsqqe,0,63e62dc1,51474e8ecf99,5.0,1900-01-01 11:03:01,19.0,20cce73f,wsqqe,台式,NaN,dinner,新北市,五股區,"(25.07080078125, 121.44287109375)",新北市,五股區,"(25.07080078125, 121.44287109375)",0.000
23,babfe111b2,wsqqm,7,7fff4be6,1a31e43c524d,6.0,1900-01-01 11:19:22,19.0,NaN,wsqqm,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中正區,"(25.02685546875, 121.53076171875)",0.000
24,07335bed02,wsqqm,8,fbbfb8a6,5ccfb41cb3a2,0.0,1900-01-01 09:21:34,17.0,NaN,wsqqt,台式,NaN,dinner,臺北市,中正區,"(25.02685546875, 121.53076171875)",臺北市,中山區,"(25.07080078125, 121.53076171875)",4.868
73,d6e844fb2e,wsqq6,22,b30567ea,a29d4390dbaa,4.0,1900-01-01 11:15:14,19.0,463204ae,wsqq7,中式,NaN,dinner,新北市,泰山區,"(25.02685546875, 121.39892578125)",新北市,新莊區,"(25.02685546875, 121.44287109375)",4.435
76,b40e3a4170,wsqqk,25,871f3be7,c339a737c9a0,2.0,1900-01-01 09:38:16,17.0,16ef5978,wsqqm,日式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)",臺北市,中正區,"(25.02685546875, 121.53076171875)",4.435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162460,5d49fef5fc,wsqqk,70710,f28be919,fc102c13f9e8,5.0,1900-01-01 11:04:17,19.0,4bc86cef,wsqq5,台式,NaN,dinner,臺北市,萬華區,"(25.02685546875, 121.48681640625)",新北市,土城區,"(24.98291015625, 121.44287109375)",6.586
162566,7fecc741bd,wsqq6,70748,f9119961,7469f786ba5f,3.0,1900-01-01 11:59:59,19.0,NaN,wsqq7,小吃,秘製雞皮,dinner,新北市,泰山區,"(25.02685546875, 121.39892578125)",新北市,新莊區,"(25.02685546875, 121.44287109375)",4.435
163531,98387041e2,wsqq0,71086,3d348369,96741259e226,6.0,1900-01-01 09:45:24,17.0,e14b373f,wsqq0,甜點,原味起士(片),dinner,桃園市,桃園區,"(24.98291015625, 121.31103515625)",桃園市,桃園區,"(24.98291015625, 121.31103515625)",0.000
164081,d2e44d1fb0,wsqqs,71338,80869d14,bbdae53722a1,6.0,1900-01-01 10:56:45,18.0,e8182792,wsqqs,日式,NaN,dinner,新北市,三重區,"(25.07080078125, 121.48681640625)",新北市,三重區,"(25.07080078125, 121.48681640625)",0.000


In [62]:
order_count_by_city = o_df_dinner.groupby('customer_city')['order_id'].count().reset_index()
order_count_by_city = order_count_by_city.rename(columns={"customer_city": "City", "order_id": "order_id_count"})

v_count_by_city = v_df_dinner.groupby('vendor_city')['vendor_id'].count().reset_index()
v_count_by_city = v_count_by_city.rename(columns={"vendor_city": "City", "vendor_id": "vendor_id_count"})

dinner_demand_supply = pd.merge(order_count_by_city, v_count_by_city, on='City', how='outer')
dinner_demand_supply['vendor_id_count'] = dinner_demand_supply['vendor_id_count'].fillna(0)


dinner_demand_supply['demand_supply_ratio'] = (dinner_demand_supply['order_id_count'] / dinner_demand_supply['vendor_id_count']).round(2)
dinner_demand_supply['demand_supply_ratio'] = dinner_demand_supply['demand_supply_ratio'].replace([np.inf, -np.inf],0)
display(dinner_demand_supply)

,City,order_id_count,vendor_id_count,demand_supply_ratio
0,三重區,1341,199.0,6.74
1,中和區,732,118.0,6.20
2,中山區,2380,216.0,11.02
3,中正區,2720,258.0,10.54
4,五股區,661,113.0,5.85
5,信義區,1983,164.0,12.09
6,內湖區,531,59.0,9.00
7,八德區,15,0.0,0.00
8,八里區,15,0.0,0.00
9,北投區,72,0.0,0.00


In [63]:
dinner_demand_supply['demand_supply_ratio'].describe()

,demand_supply_ratio
count,32.000000
mean,5.677812
std,5.430700
min,0.000000
25%,0.000000
50%,5.275000
75%,8.460000
max,24.750000


## Cross-region order in dinner
Insight;
- Cross region distance: 4.87km
- Top 3 cross region order from 中正區, 萬華區, 中山區
- Customers of these regions used to order the vendor of
中正區 -> 信義區;
萬華區 -> 新莊區, 中正區;
中山區 -> 中正區

In [64]:
cross_order = o_df_dinner[o_df_dinner['distance'] != 0]

In [65]:
cross_order['distance'].describe()

,distance
count,6844.000000
mean,4.996847
std,1.134710
min,4.432000
25%,4.435000
50%,4.868000
75%,4.868000
max,32.538000


In [69]:
cross_order.groupby('customer_city')['order_id'].count().sort_values(ascending=False).head(10)

,order_id
customer_city,
中正區,1135
萬華區,778
中山區,737
松山區,560
信義區,540
永和區,353
新莊區,347
中和區,340
五股區,321


In [68]:
cross_order.groupby('customer_city')['order_id'].count().sort_values(ascending=False).apply(lambda x: f'{(x / len(cross_order)) * 100:.2f}%').head(10)

,order_id
customer_city,
中正區,16.58%
萬華區,11.37%
中山區,10.77%
松山區,8.18%
信義區,7.89%
永和區,5.16%
新莊區,5.07%
中和區,4.97%
五股區,4.69%


In [82]:
print(len(o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中正區']))
print(len(o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中正區']))

0.5827205882352942
0.4172794117647059


/tmp/ipython-input-874899758.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中正區']))
/tmp/ipython-input-874899758.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中正區']))


In [88]:
中正區 = o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] != 0]
中正區.groupby('vendor_city')['order_id'].count().sort_values(ascending=False).head(5)

/tmp/ipython-input-3396513062.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  中正區 = o_df_dinner[o_df_dinner['customer_city'] == '中正區'][o_df_dinner['distance'] != 0]


,order_id
vendor_city,
信義區,494
中山區,289
萬華區,126
永和區,122
松山區,42


In [83]:
print(len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區']))
print(len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區']))

0.525030525030525
0.47496947496947495


/tmp/ipython-input-4043664254.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區']))
/tmp/ipython-input-4043664254.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '萬華區']))


In [89]:
萬華區 = o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] != 0]
萬華區.groupby('vendor_city')['order_id'].count().sort_values(ascending=False).head(5)

/tmp/ipython-input-872497833.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  萬華區 = o_df_dinner[o_df_dinner['customer_city'] == '萬華區'][o_df_dinner['distance'] != 0]


,order_id
vendor_city,
新莊區,242
中正區,219
中和區,145
土城區,54
中山區,51


In [85]:
print(len(o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中山區']))
print(len(o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中山區']))

0.6903361344537815
cross region order precentage; 0.3096638655462185


/tmp/ipython-input-2501634114.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] == 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中山區']))
/tmp/ipython-input-2501634114.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print('cross region order precentage;', len(o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] != 0])/len(o_df_dinner[o_df_dinner['customer_city'] == '中山區']))


In [90]:
中山區 = o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] != 0]
中山區.groupby('vendor_city')['order_id'].count().sort_values(ascending=False).head(5)

/tmp/ipython-input-1031673228.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  中山區 = o_df_dinner[o_df_dinner['customer_city'] == '中山區'][o_df_dinner['distance'] != 0]


,order_id
vendor_city,
中正區,341
松山區,98
士林區,94
信義區,82
萬華區,72
